# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Master Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by industry expert Andrew Ng, this platform is a game-changer for those looking to master deep learning. From computer vision to natural language processing, DeepLearning.AI covers it all. The interactive exercises and hands-on projects make learning engaging and practical. Whether you're a beginner or an expert, there's something for everyone. Take your skills to the next level and join the thousands of learners already benefiting from DeepLearning.AI's cutting-edge content. Start your AI journey today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Master Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by industry expert Andrew Ng, this platform is a game-changer for those looking to master deep learning. From computer vision to natural language processing, DeepLearning.AI covers it all. The interactive exercises and hands-on projects make learning engaging and practical. Whether you're a beginner or an expert, there's something for everyone. Take your skills to the next level and join the thousands of learners already benefiting from DeepLearning.AI's cutting-edge content. Start your AI journey today!

-------------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Master Deep Learning with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by industry expert Andrew Ng, this platform is a game-changer for those looking to master deep learning. From computer vision to natural language processing, DeepLearning.AI covers it all. The interactive exercises and hands-on projects make learning engaging and practical. Whether you're a beginner or an expert, there's something for everyone. Take your skills to the next level and join the thousands of learners already benefiting from DeepLearning.AI's cutting-edge content. Start your AI journey today!

-------------------------------------------------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating the feedback from all reviewers, it is clear that there are some key suggestions that overlap across the reviews:

1. Incorporate relevant keywords like "deep learning courses" and "Andrew Ng" for improved search engine visibility.
2. Enhance the title to highlight specific benefits or outcomes of the courses offered by DeepLearning.AI.
3. Ensure that the qualifications and expertise of Andrew Ng are accurately represented without any misleading information.
4. Review and substantiate any claims about the effectiveness or outcomes of the courses.
5. Check the Call-to-Action (CTA) to avoid making false promises or creating unrealistic expectations about the benefits of enrolling.

Incorporating these suggestions will not only enhance the writing for better SEO performance but also ensure ethical and legal compliance. It is essential to strike a balance between 

## Get the summary

In [15]:
print(res.summary)

Title: "Unlock Your Potential: Deep Learning Courses by Andrew Ng's DeepLearning.AI"

Discover the power of DeepLearning.AI's cutting-edge deep learning courses, designed by renowned AI expert Andrew Ng. Explore a world of possibilities in computer vision, natural language processing, and more, with hands-on projects that bring learning to life. With a focus on practical applications and real-world skills, these courses cater to beginners and experts alike. Join thousands of learners already benefiting from top-tier AI education. Empower yourself with expertise that's in demand today and tomorrow. Start your journey with DeepLearning.AI and take a step closer to mastering the future of AI.
